## Скачиваем необходимое

Сначала нужно средствами NLTK загрузить WordNet.



In [2]:
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\arbyz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Готовим данные к работе

Затем импортируем данные из подготовленного текстового файла. Файл содержит набор пар слов (только имён существительных), для которых известны экспертные оценки сходства.

Строим ассоциативный массив "пара слов -- оценка близости".

In [3]:
with open("../data/W9T1/Task_4_sample_4.csv", encoding="utf-8") as rf:
  triples = []
  for line in rf.readlines()[1:]:
    triples.append(line.strip().split(","))
  score_map = {tuple(triple[:2]): float(triple[2]) for triple in triples}

In [4]:
score_map

{('professor', 'cucumber'): 0.31,
 ('monk', 'slave'): 0.92,
 ('psychology', 'discipline'): 5.58,
 ('life', 'death'): 7.88,
 ('announcement', 'production'): 3.38,
 ('word', 'similarity'): 4.75,
 ('drink', 'car'): 3.04,
 ('precedent', 'group'): 1.77,
 ('tiger', 'cat'): 7.35,
 ('situation', 'isolation'): 3.88,
 ('dividend', 'payment'): 7.63,
 ('bird', 'cock'): 7.1,
 ('announcement', 'news'): 7.56,
 ('century', 'nation'): 3.16,
 ('cemetery', 'woodland'): 2.08,
 ('cup', 'article'): 2.4,
 ('fuck', 'sex'): 9.44,
 ('street', 'block'): 6.88,
 ('tiger', 'mammal'): 6.85,
 ('peace', 'insurance'): 2.94,
 ('smart', 'student'): 4.62,
 ('seafood', 'lobster'): 8.7,
 ('Harvard', 'Yale'): 8.13,
 ('architecture', 'century'): 3.78,
 ('peace', 'plan'): 4.75,
 ('stock', 'phone'): 1.62,
 ('president', 'medal'): 3.0,
 ('money', 'cash'): 9.15,
 ('morality', 'importance'): 3.31,
 ('Japanese', 'American'): 6.5,
 ('Arafat', 'Jackson'): 2.5,
 ('month', 'hotel'): 1.81,
 ('life', 'term'): 4.5,
 ('money', 'dollar'): 8

Отметим, что из исходного набора данных мы взяли только экспертные оценки сходства (similarity) и только для существительных. Исходный набор данных доступен по [ссылке](http://alfonseca.org/pubs/ws353simrel.tar.gz)

Посмотрим на примеры оценок. 

У слов может быть по несколько значений, которые различаются в WordNet. Здесь -- ради примера -- мы будем "жадно" выбирать первое попавшееся, но далее будем работать с ними иначе.




In [5]:
for w1, w2 in list(score_map)[:3]:
  
  print("\nWords: %s-%s\nGround truth score: %.2f" % (w1, w2, score_map[(w1, w2)]))
  
  ss1 = wn.synset(w1 + ".n.01")
  ss2 = wn.synset(w2 + ".n.01")

  print("\nPath: %.3f" % ss1.path_similarity(ss2), end=" ")
  print("\nwup: %.3f" % ss1.wup_similarity(ss2), end=" ")
  print("\nlch: %.3f" % ss1.lch_similarity(ss2), end=" ")
  print("\nshortest_path: %.3f" % ss1.shortest_path_distance(ss2))


Words: professor-cucumber
Ground truth score: 0.31

Path: 0.077 
wup: 0.500 
lch: 1.073 
shortest_path: 12.000

Words: monk-slave
Ground truth score: 0.92

Path: 0.200 
wup: 0.667 
lch: 2.028 
shortest_path: 4.000

Words: psychology-discipline
Ground truth score: 5.58

Path: 0.333 
wup: 0.875 
lch: 2.539 
shortest_path: 2.000


Вычисляем для всех пар несколько оценок

In [7]:
from itertools import product

list_pairs = list(score_map)
wup_list, true_list, path_list, lch_list = [], [], [], []

# для всех пар
for w1, w2 in list_pairs:

  try:
    all_w1 = wn.synsets(w1, pos="n")
    all_w2 = wn.synsets(w2, pos="n")

    # добавляем интересующие нас метрики и экспертные оценки
    wup = max([item1.wup_similarity(item2) \
                for item1, item2 in product(all_w1, all_w2)])
    wup_list.append(wup)

    path = max([item1.path_similarity(item2) \
                for item1, item2 in product(all_w1, all_w2)])
    path_list.append(path)

    lch = max([item1.lch_similarity(item2) \
                for item1, item2 in product(all_w1, all_w2)])
    lch_list.append(lch)

    true_list.append(score_map[(w1, w2)])

  except Exception as e:
    print(w1, w2, "error:", e)

## Вычисляем ранговую корреляцию Спирмена

In [15]:
from scipy.stats import spearmanr

coef, p = spearmanr(wup_list, true_list)
print("wup  Spearman R: %.4f" % coef)

coef, p = spearmanr(path_list, true_list)
print("path Spearman R: %.4f" % coef)

coef, p = spearmanr(lch_list, true_list)
print("lch Spearman R: %.4f" % coef)


wup  Spearman R: 0.6936
path Spearman R: 0.6535
lch Spearman R: 0.6535


In [28]:
syn = wn.synset('wood.n.01')
len(syn.hyponyms())

91

In [30]:
syn.hyponyms()[0]

Synset('alder.n.01')